In [11]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
! pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules

  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2


# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here: 
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
# load the data set ans show the first five transaction
df = pd.read_csv('https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv')
df.head(5)

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


# Get the set of product that has been purchased

In [4]:
A = (df['0'].append(df['2'])).unique()
B = set(A)
print(B)

{nan, 'Milk', 'Cheese', 'Wine', 'Bagel', 'Diaper', 'Pencil', 'Eggs', 'Meat', 'Bread'}


## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [5]:
#create an itemset based on the products
itemset = set(B)

# encoding the feature
encoded_df = []
for index, row in df.iterrows():
    rowset = set(row) 
    labels = {}
    uncommons = list(itemset - rowset)
    commons = list(itemset.intersection(rowset))
    for uc in uncommons:
        labels[uc] = 0
    for com in commons:
        labels[com] = 1
    encoded_df.append(labels)

In [7]:
  # create new dataframe from the encoded features
df1 = pd.DataFrame(encoded_df)
  # show the new dataframe
df1.head()

,NaN,Bagel,Milk,Cheese,Wine,Diaper,Pencil,Eggs,Meat,Bread
0,0,0,0,1,1,1,1,1,1,1
1,0,0,1,1,1,1,1,0,1,1
2,1,0,1,1,1,0,0,1,1,0
3,1,0,1,1,1,0,0,1,1,0
4,1,0,0,0,1,0,1,0,1,0


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

In [8]:
df2 = df1.iloc[: , 1:]
df2.head()

,Bagel,Milk,Cheese,Wine,Diaper,Pencil,Eggs,Meat,Bread
0,0,0,1,1,1,1,1,1,1
1,0,1,1,1,1,1,0,1,1
2,0,1,1,1,0,0,1,1,0
3,0,1,1,1,0,0,1,1,0
4,0,0,0,1,0,1,0,1,0


## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products. 
For this case study, we will min_support=0.2

In [13]:
df_freq = apriori(df2, min_support=0.2, use_colnames=True, verbose=1)
df_freq

Processing 141 combinations | Sampling itemset size 3


D:\Anaconda\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.425397,(Bagel)
1,0.501587,(Milk)
2,0.501587,(Cheese)
3,0.438095,(Wine)
4,0.406349,(Diaper)
5,0.361905,(Pencil)
6,0.438095,(Eggs)
7,0.476190,(Meat)
8,0.504762,(Bread)
9,0.225397,"(Bagel, Milk)"


The we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [14]:
df_ar = association_rules(df_freq, metric="confidence", min_threshold=0.6)
df_ar.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bagel),(Bread),0.425397,0.504762,0.279365,0.656716,1.301042,0.064641,1.442650
1,(Milk),(Cheese),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
2,(Cheese),(Milk),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
3,(Wine),(Cheese),0.438095,0.501587,0.269841,0.615942,1.227986,0.050098,1.297754
4,(Eggs),(Cheese),0.438095,0.501587,0.298413,0.681159,1.358008,0.078670,1.563203
5,(Meat),(Cheese),0.476190,0.501587,0.323810,0.680000,1.355696,0.084958,1.557540
6,(Cheese),(Meat),0.501587,0.476190,0.323810,0.645570,1.355696,0.084958,1.477891
7,(Eggs),(Meat),0.438095,0.476190,0.266667,0.608696,1.278261,0.058050,1.338624
8,"(Meat, Milk)",(Cheese),0.244444,0.501587,0.203175,0.831169,1.657077,0.080564,2.952137
9,"(Meat, Cheese)",(Milk),0.323810,0.501587,0.203175,0.627451,1.250931,0.040756,1.337845
